In [1]:
!pip install mysql-connector-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.8/409.8 kB 25.7 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3


In [4]:
# import libraries
import mysql.connector
import pandas as pd
# Database configuration
host = "34.101.51.177"
database = "univ"
user = "candidate"
password = "xDt905G%y23l!"
port = 3306
# Create a connection
connection = mysql.connector.connect(
    host=host,
    database=database,
    user=user,
    password=password,
    port=port
)
# Check if the connection is successful
if connection.is_connected():
    print("Connected to the database.")
else:
    print("Failed to connect to the database.")
# Create a cursor to interact with the database
cursor = connection.cursor()

Connected to the database.


In [5]:
# Execute the SHOW TABLES query
cursor.execute("SHOW TABLES")
# Fetch all table names
table_names = cursor.fetchall()
# Print the table names
print("Table Names:")
print(table_names)

Table Names:
[('course',), ('course_attendance',), ('enrollment',), ('exam',), ('exam_submission',), ('schedule',), ('student',)]


In [6]:
# Nomor 1
query = """
WITH RECURSIVE DateGenerator AS (
  SELECT id, course_id, course_days, start_dt, end_dt, start_dt AS generated_date
  FROM schedule
  UNION ALL
  SELECT d.id, d.course_id, d.course_days, d.start_dt, d.end_dt, DATE_ADD(d.generated_date, INTERVAL 1 DAY)
  FROM DateGenerator d
  WHERE d.generated_date < d.end_dt
),
baseline as
(
  SELECT id, course_id, course_days, generated_date generated_dt,
  case
  when lower(DAYNAME(generated_date)) = 'sunday' then 1
  when lower(DAYNAME(generated_date)) = 'monday' then 2
  when lower(DAYNAME(generated_date)) = 'tuesday' then 3
  when lower(DAYNAME(generated_date)) = 'wednesday' then 4
  when lower(DAYNAME(generated_date)) = 'thursday' then 5
  when lower(DAYNAME(generated_date)) = 'friday' then 6
  else 7 end as DayNum
  FROM DateGenerator
),
baseline1 as
(
  select a.* from
  (
    select *,
    case
    when id = 1 and DayNum in (2,4,6) then 1
    when id = 2 and DayNum in (3,4) then 1
    when id = 3 and DayNum in (2,5) then 1
    when id = 4 and DayNum in (3) then 1
    when id = 5 and DayNum in (2,4) then 1
    when id = 6 and DayNum in (4,5) then 1
    when id = 7 and DayNum in (2,4,6) then 1
    when id = 8 and DayNum in (2,3) then 1
    when id = 9 and DayNum in (5) then 1
    when id = 10 and DayNum in (5,6) then 1
    else 0 end as is_class
    from baseline
  ) a
  where a.is_class = 1
),
baseline2 as
(
  select id schedule_id, course_id, count(*) total_schedule
  from baseline1
  group by id, course_id
),
baseline3 as
(
  select student_id, schedule_id, count(distinct attend_dt) total_attend
  from course_attendance
  group by student_id, schedule_id
),
baseline4 as
(
  select concat(a.student_id,'-',b.course_id) student_course_id, a.schedule_id,
  a.total_attend, b.total_schedule,
  case when round((a.total_attend/b.total_schedule)*100) <75 then 1 else 0
  end as is_below_75
  from baseline3 a left join baseline2 b using(schedule_id)
),
baseline5 as
(
  SELECT
  a.id, concat(a.student_id,'-',b.course_id) student_course_id,
  c.name student_name, d.name course_name,
  case when score >= pass_threshold then 1 else 0 end as is_passed
  FROM exam_submission a
  left join exam b on a.exam_id = b.id
  left join student c on a.student_id = c.id
  left join course d on b.course_id = d.id
  order by ID asc, student_id asc, exam_id asc
)
select a.*, b.is_below_75
from baseline5 a left join baseline4 b using(student_course_id)
;
"""
cursor.execute(query)
# Fetch all rows
rows = cursor.fetchall()
df = pd.DataFrame(rows,columns=[desc[0] for desc in cursor.description])
df

,id,student_course_id,student_name,course_name,is_passed,is_below_75
0,6,2-1,Sisca,Introduction to Applied Machine Learning,1,1
1,14,13-1,Michael,Introduction to Applied Machine Learning,1,0
2,7,6-1,Horatio,Introduction to Applied Machine Learning,1,0
3,11,1-1,Anthony,Introduction to Applied Machine Learning,1,0
4,5,2-4,Sisca,Applied Data Science,1,1
5,17,13-7,Michael,Python Programming,0,1
6,16,13-3,Michael,Data Mining,1,0
7,3,2-1,Sisca,Introduction to Applied Machine Learning,0,1
8,8,6-3,Horatio,Data Mining,0,0
9,13,10-3,Kiky,Data Mining,1,0


In [7]:
# Nomor 2
query = """
with baseline as
(
  SELECT
  a.id, a.student_id, b.course_id,
  c.name student_name, d.name course_name,
  case when score >= pass_threshold then 1 else 0 end as is_passed
  FROM exam_submission a
  left join exam b on a.exam_id = b.id
  left join student c on a.student_id = c.id
  left join course d on b.course_id = d.id
)
select a.student_name, a.course_name from (
               select *, row_number()over(partition by student_id,course_id order by id asc) rn
               from baseline
              ) a
where a.rn = 2 and a.is_passed = 1
"""
cursor.execute(query)
# Fetch all rows
rows = cursor.fetchall()
df = pd.DataFrame(rows,columns=[desc[0] for desc in cursor.description])
df

,student_name,course_name
0,Sisca,Introduction to Applied Machine Learning


In [8]:
# Nomor 3
query = """
with baseline as
(
  SELECT
  a.id, a.student_id, b.course_id,
  c.name student_name, d.name course_name,
  case when score >= pass_threshold then 1 else 0 end as is_passed
  FROM exam_submission a
  left join exam b on a.exam_id = b.id
  left join student c on a.student_id = c.id
  left join course d on b.course_id = d.id
)
select a.student_name, a.course_name from (
               select *, row_number()over(partition by student_id,course_id order by id asc) rn
               from baseline
              ) a
where a.rn = 2 and is_passed = 0
"""
cursor.execute(query)
# Fetch all rows
rows = cursor.fetchall()
df = pd.DataFrame(rows,columns=[desc[0] for desc in cursor.description])
df

,student_name,course_name
0,Michael,Python Programming


In [9]:
# Nomor 4
query = """
with baseline as
(
  SELECT
  c.name student_name, d.name course_name, a.score,
  case when a.score >= b.pass_threshold then 1 else 0 end as is_passed
  FROM exam_submission a
  left join exam b on a.exam_id = b.id
  left join student c on a.student_id = c.id
  left join course d on b.course_id = d.id
),
baseline1 as
(
  select a.student_name, round(avg(a.score)) Average_Score from (select student_name, course_name, max(score) score
                from baseline
                group by student_name, course_name) a
  group by a.student_name
)
select a.ranks "Rank", a.student_name "Student Name", a.Average_Score "Average Score"
from (select *, RANK()over(order by Average_Score desc, student_name asc) ranks
     from baseline1) a
"""
cursor.execute(query)
# Fetch all rows
rows = cursor.fetchall()
df = pd.DataFrame(rows,columns=[desc[0] for desc in cursor.description])
df

,Rank,Student Name,Average Score
0,1,Anthony,81
1,2,Kiky,81
2,3,Sisca,74
3,4,Grace,73
4,5,Horatio,59
5,6,Michael,57
